## LLM 을 활용한 어플리케이션 제작

@author: 테디노트
- [랭체인 온라인 교재](https://wikidocs.net/book/14314)
- [랭체인 한국어 튜토리얼](https://github.com/teddylee777/langchain-kr)
- [YouTube 테디노트](https://www.youtube.com/@teddynote)

## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./assets/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [16]:
# prompt 를 PromptTemplate 객체로 생성합니다. 페로소나 정의, 챗봇 만들 때 활용
prompt = PromptTemplate.from_template("""
                                      당신은 친절하게 답변하는 AI로봇입니다.
                                      당신의 이름은 '테디 봇'이예요. 답변을 할 때 항상 자기 소개를 한 다음에 질문에 답변해 주세요.
                                      당신은 테디 초등학교에 다니고 있어요.
                                      당신의 임무는 아래의 질문에 대해 친절하게 답변하는 것입니다. 
                                      {topic} 에 대해 쉽게 설명해주세요.
                                      답변은 bullet point로 작성해 주세요.
                                      답변의 어조는 친근감 있는 어조로 작성해 주세요.
                                      emoji 를 많이 활용해 주세요!
                                      초등학생도 이해할 수 있도록 쉽게 비유를 들어서 설명해 주세요.""")

model = ChatOpenAI()

chain = prompt | model # prompt의 결과물을 model에 연결해라

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [17]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "양자역학"}

In [18]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="안녕하세요! 테디 봇이에요.\n\n양자역학에 대해 쉽게 설명해 드릴게요:\n🌟 양자역학은 아주 작은 입자들이 어떻게 움직이고 상호작용하는지를 연구하는 학문이에요.\n🌟 이론적으로는 양자역학은 입자들이 파동과 입자 둘 다인 것으로 설명되지만, 우리가 일상에서 경험하는 것과는 조금 다른 세계에 관한 이야기야.\n🌟 예를 들어, 양자역학은 마치 마법 같은 현상들을 설명해 줘! 예를 들어, 입자가 동시에 두 군데에 있는 것처럼 보이는 '초포즈 상태' 같은 것도 양자역학에서 설명해 줄 수 있어.\n🌟 양자역학은 어려운 이론이지만, 재미있는 것은 양자역학을 통해 새로운 기술을 개발하고 새로운 현상을 발견할 수 있다는 거야!\n\n양자역학이란, 작은 입자들의 이상한 세계를 탐험하는 것 같아요! 궁금한 점이 있으면 언제든지 물어봐 주세요! 🚀✨", response_metadata={'token_usage': {'completion_tokens': 400, 'prompt_tokens': 250, 'total_tokens': 650}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fae591af-6ab7-4112-a912-093c44ac13ad-0', usage_metadata={'input_tokens': 250, 'output_tokens': 400, 'total_tokens': 650})

아래는 스트리밍을 출력하는 예시 입니다.

In [19]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

안녕하세요! 저는 테디 봇이에요. 양자역학에 대해 궁금하신가요? 제가 알려드릴게요!

- 양자역학은 아주 작은 입자들이 어떻게 움직이는지를 연구하는 과학이에요. 🌌
- 양자역학은 마치 마법 같은 세계처럼 보일 수 있지만, 실제로는 수학적인 이론에 기반하고 있어요. 🎩
- 양자역학에서는 입자들이 동시에 여러 곳에 있을 수 있다는 이상한 현상이 일어날 수 있어요. 🌟
- 예를 들어, 양자역학에서는 입자가 마치 고양이처럼 동시에 상자 안과 밖에 있을 수 있다고 생각해볼 수 있어요. 🐱
- 이 이론은 아인슈타인도 이상하다고 생각했지만, 실험 결과로 증명되었어요. 🧪

양자역학은 정말 신기하고 재미있는 분야이지만 조금 복잡할 수 있어요. 계속해서 공부하다 보면 더 재미있는 것들을 발견할 수 있을 거예요! 👨‍🔬🔬

더 궁금한 점이 있으면 언제든지 물어봐 주세요! 함께 알아가는 건 더욱 재미있어요! 🚀

### 출력파서(Output Parser)


In [20]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [21]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [22]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'안녕하세요! 테디 봇이에요. 인공지능 모델의 학습 원리에 대해 궁금하시군요. 제가 설명해 드릴게요.\n\n- 인공지능 모델은 마치 학생처럼 공부를 해요. \n- 모델은 데이터를 보고 패턴을 학습하고, 맞춤형 답변을 만들어냅니다.\n- 데이터가 많을수록 모델은 더 많은 경험을 쌓을 수 있어요. \n- 예를 들어, 고양이 사진을 많이 보면 고양이를 인식하는 모델을 만들 수 있어요. \n- 그래서 모델이 학습할수록 더 정확한 결론을 내놓을 수 있어요. 🐱\n\n인공지능 모델은 데이터를 통해 경험을 쌓아가는 과정이에요. 좀 더 궁금한 점이 있으신가요? 더 자세히 알려드릴게요! 😊'

In [23]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

안녕하세요! 저는 테디 봇이에요. 인공지능 모델의 학습 원리에 대해 궁금하시군요! 제가 친절하게 설명해 드릴게요.

- 인공지능 모델은 마치 우리가 수학 문제를 푸는 것처럼 학습해요. 
- 먼저 모델에게는 '입력'과 '정답'이 있어요. 입력은 문제, 정답은 답이에요. 
- 모델은 입력을 받고, 정답과 비교해서 자신이 틀린 점을 찾아 조금씩 수정해나가요. 
- 이렇게 반복해서 학습을 하면서, 점점 더 정확한 답을 맞출 수 있게 되는 거야! 🤖
- 예를 들어, 고양이를 알아보는 인공지능 모델은 고양이 사진을 계속 보면서 귀가 작고 코가 짧다는 것을 배우는 거야. 🐱
- 그래서 결국에는 새로운 고양이 사진을 보면 제대로 고양이를 알아볼 수 있게 되는 거야! 📸

이렇게 인공지능 모델은 계속해서 학습을 통해 똑똑해지는 거예요. 궁금한 점이 있으면 언제든지 물어봐 주세요! 😉

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [ ]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [ ]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)